In [1]:
import os
import json
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import pprint

In [2]:
# Carla

dataset_path = '../data/record/Town05_100npcs_2'
#dataset_path = '../data/record/Town03'

with open(os.path.join(dataset_path, 'agents', '0', 'gt_vehicle_bbox', 'data.jsonl'), 'r') as f:
    vehicle_bbox_data = [json.loads(l) for l in f.readlines()]
vehicle_bbox_data = {x['frame']: x for x in vehicle_bbox_data}

try:
    with open(os.path.join(dataset_path, 'agents', '0', 'gt_location', 'data.jsonl'), 'r') as f:
        ego_location_data = [json.loads(l) for l in f.readlines()]
    ego_location_data = {x['frame']: x for x in ego_location_data}
except FileNotFoundError:
    ego_location_data = {}
    ego_pose_folder = os.path.join(dataset_path, 'agents', '0', 'ego_pose')
    for ego_pose_file in sorted(os.listdir(ego_pose_folder)):
        if not os.path.isfile(os.path.join(ego_pose_folder, ego_pose_file)) or not ego_pose_file.endswith('.json'):
            continue
        with open(os.path.join(ego_pose_folder, ego_pose_file), 'r') as f:
            frame = (int(ego_pose_file.split('.')[0]))
            ego_pose_json = json.load(f)
            ego_location_data[frame] = {
                'token': ego_pose_file['token'],
                'frame': frame,
                'timestamp': ego_pose_file['timestamp'],
                'location': ego_pose_file['translation'],
                'rotation': ego_pose_file['rotation'],  # Quaternion
            }

frames = [x['frame'] for x in ego_location_data.values()]

In [3]:
sensor_range = 100.0
# find change lane events
change_lane_events = {}
prev_actions = {}

event_actions = ['ChangeLaneLeft', 'ChangeLaneRight', 'Straight', 'Left', 'Right', 'LaneFollow', 'Unknown']
event_action_count = {action:0 for action in event_actions}

for action in event_actions:
    change_lane_events[action] = {}

for frame, data in vehicle_bbox_data.items():
    for action in event_actions:
        change_lane_events[action][frame] = []

    for vehicle in data['vehicles']:
        vid = vehicle['id']
        action = vehicle['current_action']

        event_action_count[action] += 1

        ego_location = ego_location_data[frame]['location']
        distance = np.linalg.norm(np.array(ego_location)-np.array(vehicle['location']))

        if vid in prev_actions and distance < sensor_range and action != prev_actions[vid] and action in event_actions:
            change_lane_events[action][frame].append({
                'frame': frame,
                'vehicle_id': vid,
                'event': action,
                'location': vehicle['location'],
                'rotation': vehicle['rotation'],
                'velocity': vehicle['velocity'],
            })
        
        prev_actions[vid] = action

#print('Number of events:', sum([len(x) for x in change_lane_events.values()]))

In [4]:
print(event_action_count)
for action in change_lane_events.keys():
    print(action)
    print(change_lane_events[action])

{'ChangeLaneLeft': 2907, 'ChangeLaneRight': 2796, 'Straight': 43804, 'Left': 17618, 'Right': 24708, 'LaneFollow': 15767, 'Unknown': 0}
ChangeLaneLeft
{70793: [], 70803: [], 70813: [], 70823: [], 70833: [], 70843: [], 70853: [], 70863: [], 70873: [], 70883: [], 70893: [], 70903: [{'frame': 70903, 'vehicle_id': '1205', 'event': 'ChangeLaneLeft', 'location': [45.96187973022461, 95.5119400024414, 0.003445701440796256], 'rotation': [-0.0006408690824173391, -0.0014684905763715506, -0.2327575534582138], 'velocity': [8.024054527282715, -0.03257051110267639, 1.3923644246460753e-06]}], 70913: [], 70923: [], 70933: [], 70943: [], 70953: [], 70963: [], 70973: [], 70983: [], 70993: [], 71003: [], 71013: [], 71023: [], 71033: [], 71043: [], 71053: [], 71063: [], 71073: [], 71083: [], 71093: [], 71103: [], 71113: [], 71123: [], 71133: [], 71143: [], 71153: [], 71163: [], 71173: [], 71183: [], 71193: [], 71203: [], 71213: [], 71223: [], 71233: [], 71243: [], 71253: [], 71263: [], 71273: [], 71283: [],

In [5]:
window_length = 20 * 2
window_interval = 5 * 2
frame_windows = [[x for x in frames[i:i+window_length]] for i in range(0, len(frames)-window_length, window_interval)]
print('Number of window:', len(frame_windows))

Number of window: 104


In [6]:
dbscan_eps = 2.7
min_samples = 2

clusters = {}

for action in change_lane_events.keys():
    clusters[action] = []
    for window in frame_windows:
        events = []
        features = []
        
        for frame in window:
            for event in change_lane_events[action][frame]:
                events.append(event)
                features.append(event['location'][0:2])
        if len(features) == 0:
            continue
        features = np.array(features)
        db = DBSCAN(eps=dbscan_eps, min_samples=min_samples).fit(features)
        labels = db.labels_
        cluster_ids = np.unique(labels)

        if len(cluster_ids) > 1:
            # found clusters
            for idx in cluster_ids[1:]:
                cluster_events = [events[i] for i in range(len(events)) if labels[i] == idx]
                locations = [e['location'] for e in cluster_events]
                location = np.mean(locations, axis=0).tolist()

                clusters[action].append({
                    'action': action,
                    'location': location,
                    'start_frame': window[0],
                    'end_frame': window[1],
                    'time_window': window,
                    'size': int(np.sum(labels==idx)),
                    'events': cluster_events,
                })

        # plt.scatter(features[:, 0], features[:, 1], c=labels)
        # plt.scatter([ego_location_data[f]['location'][0] for f in window], [ego_location_data[f]['location'][1] for f in window], c='red')
        # for i, event in enumerate(events):
        #     #plt.annotate(event['event'][10:11], (features[i, 0], features[i, 1]))
        #     plt.annotate(event['frame'], (features[i, 0], features[i, 1]))
        # plt.show()

print('Number of clusters:')
for action in clusters.keys():
    print(f'{action}: {len(clusters[action])}')
pprint.pprint(clusters)

with open(os.path.join(dataset_path, 'behavior_clusters.json'), 'w') as f:
    json.dump(clusters, f)


Number of clusters:
ChangeLaneLeft: 0
ChangeLaneRight: 0
Straight: 20
Left: 0
Right: 2
LaneFollow: 29
Unknown: 0
{'ChangeLaneLeft': [],
 'ChangeLaneRight': [],
 'LaneFollow': [{'action': 'LaneFollow',
                 'end_frame': 70803,
                 'events': [{'event': 'LaneFollow',
                             'frame': 70853,
                             'location': [47.34591293334961,
                                          92.00306701660156,
                                          -0.007656536065042019],
                             'rotation': [0.012672735378146172,
                                          0.0672227144241333,
                                          -0.2563781440258026],
                             'vehicle_id': '1219',
                             'velocity': [7.931687355041504,
                                          -0.0427006334066391,
                                          1.535415663056483e-06]},
                            {'event': 'LaneFo